# Import libraries

In [ ]:
import numpy as np
import pandas as pd
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
import kmapper as km
import networkx as nx
import scikit_posthocs as sp
import matplotlib.pyplot as plt
import seaborn as sns
import math
import dyneusr as dyn
import matplotlib.style as style
import warnings
import tqdm

from tda.tda_graph import TDAGraph
from tda.tda_graph_visualizer import TDAVisualizer
from community_detection.community import Community
from tda.tda_mapping import MappingTDA
from harmonization.neurocombat_pyClasse import *
from stats.stats_test import ChiSquaredAnalyzer, FeatureSetStatisticsAnalyzer, ContingencyTableComparator
from stats.pdf_estimation import PDFEstimator
from tda.tda_mapping import MappingTDA
from graph_metrics.graph_metrics import GraphMetrics
from scipy.spatial.distance import cosine 


from scipy import stats
from scipy.spatial.distance import jensenshannon, euclidean, cityblock, correlation
from statsmodels.stats.multitest import multipletests
from statsmodels.graphics.mosaicplot import mosaic
from itertools import combinations
from tuning.heatmap_creation import Heatmap

from collections import defaultdict, Counter
from sklearn.metrics import adjusted_rand_score, accuracy_score, f1_score, mutual_info_score
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler

import numpy as np
from scipy import stats
from scipy.spatial.distance import cosine
from tqdm import tqdm

# Set random seed for reproducibility
seed_value = 42

# Load data

In [ ]:
data_path = '/path/to/your/data'  # Update this path to your data directory

data = pd.read_csv(f'{data_path}/your_data_file.csv')  # Update this filename to your actual dataset
labels = pd.read_csv(f'{data_path}/your_labels_file.csv')  # Update this filename to your actual labels file

data_2 = pd.read_csv(f'{data_path}/your_data_2_file.csv')  # Update this filename to your actual second dataset
labels_2 = pd.read_csv(f'{data_path}/your_labels_2_file.csv')  # Update this filename to your actual second labels file

In [ ]:
features_name = data.columns
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)
data_scaled_2 = scaler.fit_transform(data_2)

data_scaled = pd.DataFrame(data_scaled, columns=features_name)
data_scaled_2 = pd.DataFrame(data_scaled_2, columns=features_name)

# TDA

In [ ]:
tdagraph = TDAGraph(data_scaled,
                    filter_func='tsne',
                    n_cubes=30,
                    perc_overlap=0.8,
                    cluster_algorithm='dbscan',
                    min_samples=3,
                    eps=8,
                    metric='chebyshev',)

graph, bins, lens, cover = tdagraph.create_graph()

In [ ]:
save_path = '/Users/stefanovannoni/Desktop/PhD/PROGETTI/FINAL PROJECT TDA/graphs'
tdagraph_visualizer = TDAVisualizer(graph,
                                    path_html='graph_disc.html',
                                    output_path=save_path)

tdagraph_visualizer.visualize(type_='dyn', target=labels, template=None)

# Community detection

In [ ]:
community = Community(graph,
                      algorithm='louvain',
                      seed_value=seed_value,
                      dataset=data,
                      labels_df=labels)

In [ ]:
comms = community.community_detection(k=10)
community.compute_modularity()
print(f"Number of communities detected: {len(comms)}")

In [ ]:
samples_df = community.get_sample_idx()

filtered_samples_df = community.communities_filtering(samples_df)

valid_ids = labels[labels['Study Group'].isin(['HC', 'ROP', 'ROD'])].index
filtered_samples_small_df = []
for df in filtered_samples_df:
    filtered_df = df[df.index.isin(valid_ids)]
    filtered_samples_small_df.append(filtered_df)

In [ ]:
study_group_frequency_filtered = community.get_labels_frequency(filtered_samples_df)
study_group_frequency_filtered

# TDA Mapping

In [ ]:
mappingtda = MappingTDA(
    lens=lens,
    cover=cover,
    bins=bins,
)

communities = community.get_node_idx_communities(merging=False)
for n in range(len(communities)):
    print(f'Number of nodes in community {n}: {len(communities[n])}')

print(f'Total number of nodes in the graph: {len(graph["nodes"])}')

In [ ]:
contingency_table_small, comm_overlap_replication = mappingtda.build_contingency_table(
    data=data_scaled_2,
    labels=labels_2,
    contigency_table_constructed=study_group_frequency_filtered,
    tdagraph=tdagraph,
    communities=community,
    return_overlap=True,
    plot_overlap=True,
)

contingency_table_small